In [2]:
#!/usr/bin/env python3.7

# Copyright 2022, Gurobi Optimization, LLC

# We find alternative epsilon-optimal solutions to a given knapsack
# problem by using PoolSearchMode

from __future__ import print_function
import gurobipy as gp
from gurobipy import GRB
import sys

In [3]:
    # Sample data
    Groundset = range(10)
    objCoef = [32, 32, 15, 15, 6, 6, 1, 1, 1, 1]
    knapsackCoef = [16, 16,  8,  8, 4, 4, 2, 2, 1, 1]
    Budget = 33

In [4]:
    # Create initial model 创建模型
    model = gp.Model("poolsearch")

    # Create dicts for tupledict.prod() function
    objCoefDict = dict(zip(Groundset, objCoef))
    knapsackCoefDict = dict(zip(Groundset, knapsackCoef))

    # Initialize decision variables for ground set: 添加变量
    # x[e] == 1 if element e is chosen
    Elem = model.addVars(Groundset, vtype=GRB.BINARY, name='El')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-14


In [5]:
    # Set objective function 设置目标函数
    # Gurobi 模型对象。常用的方法包括 AddVar（向模型添加新的决策变量）、 AddConstr（向模型添加新的约束）、 
    # Optimize（优化当前模型）和Get （检索属性的值）。
    model.ModelSense = GRB.MAXIMIZE
    model.setObjective(Elem.prod(objCoefDict))

In [6]:
    # Constraint: limit total number of elements to be picked to be at most
    # Budget
    model.addConstr(Elem.prod(knapsackCoefDict) <= Budget, name='Budget')

    # Limit how many solutions to collect 想要收集多少个解
    model.setParam(GRB.Param.PoolSolutions, 1024)
    
    # Limit the search space by setting a gap for the worst possible solution that will be accepted
    model.setParam(GRB.Param.PoolGap, 0.10)

    # do a systematic search for the k-best solutions
    # 选择不同的模式来探索 MIP 搜索树。在默认设置 ( PoolSearchMode=0) 下，MIP 求解器尝试找到模型的最优解。它保留了沿途发现的其他解决方案，但这些都是偶然的。
    # 通过将此参数设置为非默认值，MIP 搜索将在找到最佳解决方案后继续，以找到其他高质量的解决方案。
    # 使用非默认值 (PoolSearchMode=1或 PoolSearchMode=2)，MIP 求解器将尝试找到n 解决方案，其中n由 PoolSolutions参数的值确定。
    # 设置为 1 时，无法保证额外解决方案的质量，而设置为 2 时，求解器将找到n最佳解决方案。求解的成本将随着此参数值的增加而增加。
    model.setParam(GRB.Param.PoolSearchMode, 2)

Set parameter PoolSolutions to value 1024
Set parameter PoolGap to value 0.1
Set parameter PoolSearchMode to value 2


In [8]:
    # save problem
    model.write('poolsearch.lp')

    # Optimize
    model.optimize()
    model.setParam(GRB.Param.OutputFlag, 0)

In [1]:
    # Status checking
    status = model.Status
    if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
        print('The model cannot be solved because it is infeasible or '
              'unbounded')
        sys.exit(1)

    if status != GRB.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)

NameError: name 'model' is not defined

In [10]:
    # Print best selected set
    print('Selected elements in best solution:')
    print('\t', end='')
    for e in Groundset:
        if Elem[e].X > .9:
            print(' El%d' % e, end='')
    print('')

Selected elements in best solution:
	 El0 El1 El8


In [11]:
    # Print number of solutions stored
    nSolutions = model.SolCount
    print('Number of solutions found: ' + str(nSolutions))

Number of solutions found: 21


In [12]:
    # Print objective values of solutions
    for e in range(nSolutions):
        model.setParam(GRB.Param.SolutionNumber, e)
        print('%g ' % model.PoolObjVal, end='')
        if e % 15 == 14:
            print('')
    print('')

65 65 64 63 63 63 63 62 62 60 60 60 60 60 60 
60 60 59 59 59 59 


In [13]:
    # print fourth best set if available
    if (nSolutions >= 4):
        model.setParam(GRB.Param.SolutionNumber, 3)

        print('Selected elements in fourth best solution:')
        print('\t', end='')
        for e in Groundset:
            if Elem[e].Xn > .9:
                print(' El%d' % e, end='')
        print('')

Selected elements in fourth best solution:
	 El1 El2 El3 El8
